In [1]:
from keras.applications.resnet50 import ResNet50
# from keras.applications.VGG16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import matplotlib.pyplot as plt
import h5py
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.callbacks import TensorBoard, ModelCheckpoint
import datetime

import numpy as np

def get_file_name ():
    now = datetime.datetime.now()
    return "%d%d%d-%d%d" % (now.year, now.month, now.day, now.hour, now.minute)

def get_model ():
    base_model = ResNet50(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(nr_labels, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    
    for layer in model.layers[:-5]:
        layer.trainable = False

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def get_tensor_board () :
    return TensorBoard(log_dir='./logs/' + get_file_name()  + '/', histogram_freq=0,
                          write_graph=False, write_images=False)
def get_generator (folder, batch_size=32):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_generator = train_datagen.flow_from_directory(
        folder,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')
    
    return train_generator

Using TensorFlow backend.


In [ ]:
import scipy.io
labelMat = scipy.io.loadmat('utils/imagelabels.mat')

#distancemap = scipy.io.loadmat('distancematrices17.mat')
labels = labelMat['labels']
nr_labels = max(max(labels))
batch_size = 32
nr_epochs = 100
num_samples = 1020


In [ ]:

tensorboard = get_tensor_board()
model = get_model()


train_generator = get_generator('data4/train', batch_size=batch_size)
valid_generator = get_generator('data4/valid', batch_size=batch_size)


model.fit_generator(train_generator, 
                    epochs=nr_epochs,
                    steps_per_epoch= num_samples / batch_size, 
                    validation_data=valid_generator,
                    validation_steps = num_samples / batch_size,
                    callbacks=[
                        tensorboard,
                        ModelCheckpoint('models/model.h5', verbose=1, save_best_only=True)
                    ])

Instructions for updating:
Colocations handled automatically by placer.


c:\users\axel\kamai\flower-image-classification\venv\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 1020 images belonging to 102 classes.
Found 1020 images belonging to 102 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
11/31 [=========>....................] - ETA: 1:12 - loss: 4.7311 - acc: 0.0966

In [ ]:
model.evaluate_generator(generator=valid_generator, steps=1)

In [ ]:
print(model.metric_names)